# 1 - Loading libraries

In [1]:
library(GenomicFeatures)
library(GenomicRanges)
library(SummarizedExperiment)
library(rtracklayer)
library(consensusDE)

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors

Loading required package: stats4


Attaching package: ‘S4Vectors’


The following object is masked from ‘package:utils’:

    findMatches


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomeInfoDb

Loading required package: GenomicRanges

Loa

# 2 - Make a txdb object from a GTF file

In [2]:
gtf_path <- "/home/jovyan/Data4Docker/gencode.v29.annotation.gtf"
txdb <- makeTxDbFromGFF(gtf_path, format="gtf")
genes <- genes(txdb)

Import genomic features from the file as a GRanges object ... 
OK

Prepare the 'metadata' data frame ... 
OK

Make the TxDb object ... 
Warning message in .get_cds_IDX(mcols0$type, mcols0$phase):
"The "phase" metadata column contains non-NA values for features of type
  stop_codon. This information was ignored."
OK



# 3 - Load RNA-seq count data

In [4]:
counts_path <- "/home/jovyan/Data4Docker/final_output.tsv"
counts <- as.matrix(read.delim(counts_path, header=TRUE, row.names=1,check.names=FALSE))

# 4 - Filter txdb object

In [5]:
# Assuming the row names of your counts matrix are gene identifiers
# And assuming gene_id in mcols(gene_info) can be matched to them
valid_genes <- rownames(counts) %in% mcols(genes)$gene_id

# Filter counts matrix to keep only rows for valid genes
counts <- counts[valid_genes, ]

# Filter gene_info based on the filtered counts
filtered_gene_info <- genes[rownames(counts) %in% mcols(genes)$gene_id]

# Ensure the order matches the counts matrix
filtered_gene_info <- filtered_gene_info[match(rownames(counts), mcols(filtered_gene_info)$gene_id)]
rownames(counts) <- mcols(filtered_gene_info)$gene_id
colnames(counts) <- rownames(colData)

# 5 - Load metadata

In [6]:
metadata_path <- "/home/jovyan/Data4Docker/PD_metadata_filtered.csv"
metadata <- read.csv(metadata_path, header=TRUE,row.names=1)
colData = DataFrame(metadata)

# 6 - Create RangedSummarizedExperiment

In [7]:
# Create the RangedSummarizedExperiment object
rse <- SummarizedExperiment(rowRanges = filtered_gene_info,
                            colData = metadata,
                            assays = list(counts = counts))

# 7 - Filter rse object 

In [8]:
# Filter out samples where 'group' contains "Other" or is an empty string
filtered_samples <- (grepl("Other", colData(rse)$group, ignore.case = TRUE) | colData(rse)$group == "")

# Filter out these samples from the SummarizedExperiment object
rse <- rse[, !filtered_samples]

# 6 - Perform differential expression analysis

Columns of interest: group
Created paired groups to make the analysis sustainable, otherwise the kernel may die

## 6.1 - Load annotation libraries to attach gene symbol data to the DE results

In [9]:
library(org.Hs.eg.db)
library(EnsDb.Hsapiens.v86)



Loading required package: ensembldb

Loading required package: AnnotationFilter


Attaching package: 'ensembldb'


The following object is masked from 'package:stats':

    filter




## 6.2 - Filter features with low-counts

In [10]:
rse <- buildSummarized(summarized = rse,
                       filter = TRUE,
                       tx_db = txdb)

colData(rse)$file <- rownames(colData(rse)) # I added a feature named file in the colData DF

Warning message in buildSummarized(summarized = rse, filter = TRUE, tx_db = txdb):
"No output directory provided. The se file and sample_table will not
          be saved"
Warning message in buildSummarized(summarized = rse, filter = TRUE, tx_db = txdb):
"The summarized experiment provided does not include a "file"
            column. This will create errors when running the DE analysis. Update
            the summarized experiment with the experimental details before
            processing to DE analysis"
A txdb was provided as input. Meta-data has been updated



## 6.3 - Declare functions to perform DEA and write the results

In [21]:
performDEAnalysis <- function(rse, month) {
  # Define the regex pattern based on the month
  regex_pattern <- if(month == "M0") "M0(\\D|$)" else month
  
  # Filter the SummarizedExperiment object for the current month
  selected_samples <- grep(regex_pattern, colData(rse)$group, value = TRUE)
  rse_filtered <- rse[, colData(rse)$group %in% selected_samples]

   # Display the groups and the number of samples being compared
  group_counts <- table(colData(rse_filtered)$group, colData(rse_filtered)$Month)
  print("Groups and the number of samples being compared:")
  print(group_counts)
  
  # Perform differential expression analysis
  de_result <- multi_de_pairs(summarized = rse_filtered,
                              paired = "unpaired",
                              ruv_correct = FALSE,
                               ensembl_annotate = org.Hs.eg.db)
  
  # Print confirmation message
  print(paste("The RNA-seq data for the month", month, "has been successfully processed."))
  
  # Return the DE analysis result
  return(de_result)
}

writeDEResults <- function(de_result) {
  # Infer contrast name from the names of the merged component
  contrast_name <- names(de_result$merged)[1]
  
  # Write specific portions of the DE result to disk
  write.csv(de_result$merged[[contrast_name]], 
            paste0("/home/jovyan/Data4Docker/Merged_", contrast_name, ".csv"))
  write.csv(de_result$deseq['full_results']$full_results[[contrast_name]], 
            paste0("/home/jovyan/Data4Docker/deseq_", contrast_name, ".csv"))
  write.csv(de_result$edger['full_results']$full_results[[contrast_name]]$table, 
            paste0("/home/jovyan/Data4Docker/edger_", contrast_name, ".csv"))
  write.csv(de_result$voom[['full_results']][[contrast_name]], 
            paste0("/home/jovyan/Data4Docker/voom_", contrast_name, ".csv"))
}

# 7 - Executing the workflow

In [22]:
# Define the vector of time points
time_points <- c("M0", "M06", "M12", "M24", "M36")

# Loop through each time point
for(month in time_points) {
  # Perform DE analysis for the current month
  de_result <- performDEAnalysis(rse, month)
  
  # Write the DE analysis results to disk
  writeDEResults(de_result)
  
  # Print message indicating completion of processing for the current month
  cat("Completed processing for month:", month, "\n")
}

[1] "Groups and the number of samples being compared:"
            
              M0
  Case_M0    206
  Control_M0 250


converting counts to integer mode

using supplied model matrix

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 2783 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing

'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns



[1] "The RNA-seq data for the month M0 has been successfully processed."
Completed processing for month: M0 
[1] "Groups and the number of samples being compared:"
             
              M06
  Case_M06    140
  Control_M06  87


converting counts to integer mode

using supplied model matrix

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 4751 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing

'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns



[1] "The RNA-seq data for the month M06 has been successfully processed."
Completed processing for month: M06 
[1] "Groups and the number of samples being compared:"
             
              M12
  Case_M12     82
  Control_M12  37


...

[1] "The RNA-seq data for the month M12 has been successfully processed."
Completed processing for month: M12 
[1] "Groups and the number of samples being compared:"
             
              M24
  Case_M24     98
  Control_M24  57


Warning message in multi_de_pairs(summarized = rse_filtered, paired = "unpaired", :
"sample_table provided contained groups with less than 2 replicates!"
Warning message in multi_de_pairs(summarized = rse_filtered, paired = "unpaired", :
"Length	Do NOT contain biological replicates!"
converting counts to integer mode

using supplied model matrix

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 1947 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing

'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns



[1] "The RNA-seq data for the month M24 has been successfully processed."
Completed processing for month: M24 
[1] "Groups and the number of samples being compared:"
             
              M36
  Case_M36      6
  Control_M36   5


Warning message in multi_de_pairs(summarized = rse_filtered, paired = "unpaired", :
"sample_table provided contained groups with less than 2 replicates!"
Warning message in multi_de_pairs(summarized = rse_filtered, paired = "unpaired", :
"Length	Do NOT contain biological replicates!"
converting counts to integer mode

using supplied model matrix

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns



[1] "The RNA-seq data for the month M36 has been successfully processed."
Completed processing for month: M36 
